In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('data/combined_data_1.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
df2 = pd.read_csv('data/combined_data_2.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
# df3 = pd.read_csv('data/combined_data_3.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])
# df4 = pd.read_csv('data/combined_data_4.txt', header=None, names=['cust_id', 'rating'], usecols=[0,1])

df = df1.append([df2])
df.shape

(51040565, 2)

In [3]:
movies_data = pd.read_csv('data/movie_titles.csv', encoding="ISO-8859-1", header=None, names=['movie_id', 'year', 'name'])
movies_data.shape

(17770, 3)

In [4]:
df.index = np.arange(0, len(df))

In [5]:
df_nan = pd.DataFrame(pd.isnull(df.rating))
df_nan = df_nan[df_nan['rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i, j in zip(df_nan['index'][1:], df_nan['index'][:-1]):
    if i - j - 1 < 0:
        print(i, j)
    temp = np.full((1, i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

last_rec = np.full((1, len(df) - df_nan.iloc[-1, 0] - 1), movie_id)
movie_np = np.append(movie_np, last_rec)

len(movie_np)

51031355

In [7]:
df = df[pd.notnull(df['rating'])]
df['movie_id'] = movie_np.astype(int)
df['cust_id'] = df['cust_id'].astype(int)
df.shape

(51031355, 3)

In [8]:
df.head()

,cust_id,rating,movie_id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [14]:
f = ['count', 'mean']
df_movie_summ = df.groupby('movie_id')['rating'].agg(f)
df_movie_summ.index = df_movie_summ.index.map(int)
movie_thresh = round(df_movie_summ['count'].quantile(0.7), 0)
drop_movie_list = df_movie_summ[df_movie_summ['count'] < movie_thresh].index

df_cust_summ = df.groupby('cust_id')['rating'].agg(f)
df_cust_summ.index = df_cust_summ.index.map(int)
cust_thresh = round(df_cust_summ['count'].quantile(0.7), 0)
drop_cust_list = df_cust_summ[df_cust_summ['count'] < cust_thresh].index

In [15]:
print(f'Original shape: {df.shape}')
df_cleaned = df[~df['movie_id'].isin(drop_movie_list)]
df_cleaned = df_cleaned[~df_cleaned['cust_id'].isin(drop_cust_list)]
print(f'New shape: {df_cleaned.shape}')
df_cleaned.head()

Original shape: (51031355, 3)
New shape: (36695974, 3)


,cust_id,rating,movie_id
696,712664,5.0,3
697,1331154,4.0,3
698,2632461,3.0,3
699,44937,5.0,3
700,656399,4.0,3


In [16]:
# Matrix
df_piv = pd.pivot_table(df_cleaned, values='rating', index='cust_id', columns='movie_id')
print(df_piv.shape)

(143567, 2765)


In [19]:
movies_data.set_index('movie_id', inplace=True)
movies_data.head()

,year,name
movie_id,,
1,2003.0,Dinosaur Planet
2,2004.0,Isle of Man TT 2004 Review
3,1997.0,Character
4,1994.0,Paula Abdul's Get Up & Dance
5,2004.0,The Rise and Fall of ECW


In [20]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [21]:
reader = Reader()
data = Dataset.load_from_df(df_cleaned[['cust_id', 'movie_id', 'rating']][:], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

KeyboardInterrupt: 